# El algoritmo de Mapper

El algoritmo de Mapper nos permite encontrar una representación de los datos que nos permita visualizarlos y entenderlos mejor. Para ello, el algoritmo de Mapper se basa en la idea de que los datos se pueden representar como una red de nodos conectados por aristas. 


Basado en una función crearemos una "proyección" (dependiendo de lo que queramos estudiar) de los datos, luego utilizaremos las preimágenes de una cubierta abierta de la imágen (de la proyección). Realizaremos un clustering de los datos en mis preimágenes y luego, con la relación de nervio en los clusters, creamos un grafo que nos permitirá visualizar los datos.

Para ello, utilizaremos la librería [KeplerMapper](https://kepler-mapper.scikit-tda.org/), que nos permite realizar el algoritmo de Mapper de forma sencilla. También la librería [giotto-tda](https://giotto-ai.github.io/gtda-docs/latest/) cuenta con una implementación de Mapper. En este notebook nos centraremos en `KeplerMapper`, ustedes pueden probar con `giotto-tda` si lo desean.

La idea de la visualización dada de Mapper es que podamos inferir relaciones (basadas en la proyección) entre los datos.

Para entender mejor el algoritmo de Mapper, estudiaremos una base de datos de acciones bursátiles. Cabe mencionar que, la elección de los parámetros de Mapper depende del problema que estemos trabajando, y no hay un óptimo como tal, todo depende de lo que queramos estudiar. 

Con esta base de datos, queremos estudiar aquellas acciones que se parezcan entre ellas, es decir, que tengan un comportamiento similar en el mercado.



In [21]:
import yfinance as yf
import kmapper as km
from kmapper.jupyter import display
import umap #Uniform Manifold Approximation and Projection for Dimension Reduction
import sklearn
import sklearn.manifold as manifold #https://scikit-learn.org/stable/modules/manifold.html
import matplotlib.pyplot as plt
import numpy as np



El índice Standard & Poor's 500 (Standard & Poor's 500 Index), también conocido como S&P 500, es uno de los índices bursátiles más importantes de Estados Unidos. Al S&P 500 se lo considera el índice más representativo de la situación real del mercado. Se trabajará un archivo de ticketmaster.


# `Obtengamos los datos`

El archivo `SP500_tickernames.txt` contiene las etiquetas de algunas companías que cotizan acciones, utilizaremos estas etiquetas para obtener los datos de las acciones de estas compañías en un periódo actual.


In [2]:
# Leer el archivo
filename = open("SP500_tickernames.txt", "r")
raw_tickernames = filename.read()
ticker_names = raw_tickernames.split("\n")
ticker_names = ticker_names[:len(ticker_names)-1]

In [22]:
# Definir el rango de fechas
start_date_string = "2020-01-01"
end_date_string = "2022-04-02"

# Obtengamos los datos históricos
raw_data = yf.download(ticker_names, start=start_date_string, end=end_date_string)

[*********************100%***********************]  495 of 495 completed

16 Failed downloads:
- VIAC: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- ANTM: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- DISCK: No timezone found, symbol may be delisted
- INFO: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- NLOK: No timezone found, symbol may be delisted
- FBHS: No timezone found, symbol may be delisted


In [24]:
# obtenga precios de cierre diarios y elimine las columnas que faltan
df_close = raw_data['Adj Close'].dropna(axis='columns')
df_close.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00,84.108368,28.982893,148.484299,73.449387,76.495049,81.061348,168.809998,82.122177,199.948456,334.429993,...,142.405029,57.058815,59.038685,54.990803,76.823212,96.286095,141.331955,259.140015,47.579472,131.088669
2020-01-03 00:00:00,82.757935,27.548195,148.493637,72.735329,75.768967,80.041870,166.820007,81.121025,199.615448,331.809998,...,140.292755,57.333221,58.564056,54.382256,77.275398,95.984512,140.962463,256.049988,46.908058,131.108231
2020-01-06 00:00:00,83.002586,27.219410,146.053162,73.314880,76.366913,81.213783,179.039993,81.546043,198.311935,333.709991,...,140.015091,57.250896,59.013714,54.681705,76.775108,95.927986,140.147858,258.010010,46.336456,130.101669
2020-01-07 00:00:00,83.257011,27.119778,144.320648,72.970085,75.931267,80.632591,180.350006,81.092674,194.030411,333.390015,...,140.679504,57.131989,58.530735,54.971481,76.486496,96.097626,140.024719,256.470001,46.055180,130.541397
2020-01-08 00:00:00,84.079010,27.737495,142.662643,74.143898,76.469421,81.413864,178.690002,81.423256,194.410995,337.869995,...,141.562103,57.077106,57.648079,55.551044,76.746239,96.267250,141.701385,247.639999,46.508842,130.258011


In [25]:
print('Hay ' + str(df_close.shape[0]) + ' compañías')
print('El rango de fechas cuenta con '+ str(df_close.shape[1]) + ' días')

Hay 568 compañías
El rango de fechas cuenta con 478 días


Para aplicar el algoritmo de Mapper de manera adecuada, convertiremos esta base de datos en una matriz de datos, donde cada fila corresponde a una acción y cada columna corresponde a un día. De igual manera estandarizaremos los datos para que todos tengan la misma escala (esto con el objeto de que nuestros no estén tan dispersos).

In [26]:
# convierta pandas dataframe a numpy array, estandaricemos los datos y transpongamos el array
data = df_close.to_numpy()
data = data-np.mean(data, axis=0)/np.std(data, axis=0)
data = data.transpose()

El razón de retorno de una inversión es la ganancia o pérdida que se obtiene en una inversión en relación con la cantidad invertida inicialmente. El porcentaje de retorno se calcula dividiendo la ganancia o pérdida neta por el costo original de la inversión. Esta tasa de rendimiento permite a los inversores comparar el éxito de una inversión con otras.

In [27]:
# calcula el porcentaje de retorno (de la inversión) de cada ticker sobre el rango de fechas
per_return = (df_close.to_numpy().transpose()[:,504] - df_close.to_numpy().transpose()[:,0])/df_close.to_numpy().transpose()[:,0]

# Aplicando el algoritmo mapper


Primero inicializaremos el algoritmo. Para ello, utilizaremos la función `km.KeplerMapper()`, donde `km` es el alias de `KeplerMapper`.

En un principio puedes escribir los parametros deseados aquí, pero para que sea más didáctico lo haremos paso a paso.

In [28]:
# initializa mapper
mapper = km.KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


Para este caso particular usaremos una proyección dada por el isomap, que es un algoritmo de reducción de dimensionalidad. Intuitivamente, el isomap encuentra una representación de los datos en un espacio de menor dimensión, donde la distancia entre dos puntos es la distancia geodésica (distancia que minimiza la energía) a lo largo de los puntos más cercanos en los datos originales.

Notemos que a nuestra proyección le pedimos `n_components=500`, lo cual indica que la cubierta en la imágen tendrá 500 abiertas. Dependiente la cantidad de componentes que pidamos, nuestro gráfo cambia de complejidad (más componentes, más nodos y aristas, o menos, o inexistencia).

El UMAP sirve para dar cierta categorización a los datos cuando no son claras las categorías, el UMAP es un algoritmo avanzado que necesita de muchos requerimientos y que no son parte del curso, pero que es importante mencionar para este ejemplo. No nos centraremos mucho en lo que hace UMAP, pero quedense con la idea de la categorización.

In [29]:
# proyecta los datos a dos dimensiones usando dos transformaciones:
# Coordenada 1 isomap https://en.wikipedia.org/wiki/Isomap
# Coordenada 2 UMAP https://arxiv.org/abs/1802.03426
projected_data = mapper.fit_transform(data, projection=[manifold.Isomap(n_components=500, n_jobs=-1), umap.UMAP(n_components=2,random_state=1)])


..Composing projection pipeline of length 2:
	Projections: Isomap(n_components=500, n_jobs=-1)
		UMAP(random_state=1)
	Distance matrices: False
False
	Scalers: MinMaxScaler()
MinMaxScaler()
..Projecting on data shaped (478, 568)

..Projecting data using: 
	Isomap(n_components=500, n_jobs=-1)


..Scaling with: MinMaxScaler()

..Projecting on data shaped (478, 568)

..Projecting data using: 
	UMAP(random_state=1, verbose=1)

UMAP(random_state=1, verbose=1)
Mon May  1 17:49:58 2023 Construct fuzzy simplicial set
Mon May  1 17:49:59 2023 Finding Nearest Neighbors
Mon May  1 17:49:59 2023 Finished Nearest Neighbor Search
Mon May  1 17:49:59 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Mon May  1 17:50:04 2023 Finished embedding

..Scaling with: MinMaxScaler()



Una vez obtenida la proyección y el número de abiertos en la cubierta de la imágen. Procedemos a pedir a mapper que en nuestra proyección, nos haga un clustering de los datos y nos cree el gráfo. Para ello, utilizaremos la función `mapper.map()`:
    
    1. `projected_data`: la proyección que queremos estudiar.
    2. `data`: los datos originales.
    3. `clusterer`: el algoritmo de clustering que queremos utilizar.

En este caso usamos la proyección descrita anteriormente, y como clusterer usaremos `DBSCAN` con la métrica del coseno. La métrica del coseno nos indica que tan parecidos son dos vectores o dos pendientes. Como queremos conocer el comportamiento de las acciones (sus ascensos y descensos) esta métrica es ideal.

In [30]:
# clusterizar los datos using DBSCAN
# Checar los distintos métodos de clusterización https://scikit-learn.org/stable/modules/clustering.html
G = mapper.map(projected_data, data, clusterer=sklearn.cluster.DBSCAN(metric="cosine"))

Mapping on data shaped (478, 568) using lens shaped (478, 2)

Creating 100 hypercubes.

Created 20 edges and 26 nodes in 0:00:00.105634.


In [32]:
G['meta_data']

{'projection': 'UMAP(random_state=1)',
 'n_cubes': 10,
 'perc_overlap': 0.1,
 'clusterer': "DBSCAN(metric='cosine')",
 'scaler': 'MinMaxScaler()'}

El resultado del algoritmo, dice que creo 100 hypercubos (o cajas de dimensiones apropiadas) para cubrir la base de datos. De allí tenemos 26 nodos, la relación entre los nodos del gráfo es que si tienen un traslape del 1% (esto se puede cambiar) entonces se conectan por una arista.

Para revisar estos datos pueder checar el comando `G['meta_data'].

In [33]:
#definir un nombre de archivo para guardar los parámetros de Mapper
fileID = 'projection=' + G['meta_data']['projection'].split('(')[0] + '_' + \
'n_cubes=' + str(G['meta_data']['n_cubes']) + '_' + \
'perc_overlap=' + str(G['meta_data']['perc_overlap']) + '_' + \
'clusterer=' + G['meta_data']['clusterer'].split('(')[0] + '_' + \
'scaler=' + G['meta_data']['scaler'].split('(')[0]



Una vez que ya aplicamos el algoritmo a nuestros datos, vamos a visualizarlos. Para ello, utilizaremos la función `mapper.visualize()`:

    1. `G`: el gráfo que queremos visualizar.
    2. `title`: el título que queremos que tenga el archivo html.
    3. `custom_tooltips:`: los datos que queremos que aparezcan en cada nodo del gráfo.
    4. `color_values`: el color que queremos que tengan los nodos del gráfo.
    5. `color_function_name`: la función que queremos que nos de el color de los nodos.
    6. `node_color_function`: el color que clasifique a los elementos de los nodos.

`kepplermapper.visualize()` nos crea un archivo html que contiene el gráfo que queremos visualizar.

Para visualizar el gráfo, le pedimos a `km.jupyter.display` que nos muestre el archivo html que se generó.

In [39]:
#Visualicemos la gráfica
mapper.visualize(G, 
                title=fileID,
                custom_tooltips = df_close.columns.to_numpy(),
                color_values = np.log(per_return+1),
                color_function_name = 'Log Percentage Return',
                node_color_function=np.array(['average','std','sum','max','min']))

km.jupyter.display("mapper_example_" + fileID + ".html")
#km.draw_matplotlib(G)
#plt.show()

Wrote visualization to: mapper_visualization_output.html


c:\Users\L03088338\AppData\Local\Programs\Python\Python310\lib\site-packages\kmapper\visuals.py:344: RuntimeWarning: invalid value encountered in longlong_scalars
  height = np.floor(((bar / max_bucket_value) * 100) + 0.5)
c:\Users\L03088338\AppData\Local\Programs\Python\Python310\lib\site-packages\kmapper\visuals.py:345: RuntimeWarning: invalid value encountered in longlong_scalars
  perc = round((bar / sum_bucket_value) * 100.0, 1)


C:\Users\L03088338\AppData\Roaming\Python\Python310\site-packages\IPython\core\display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


En teoría, si todo salió bien, cada componente conexa del gráfo debería contener información de comportamiento similar en los elementos del nodo. 

Para determinar esto, estudiemos uno de los nodos. Para ello, listaremos el id de los nodos que se encuentran en el gráfo. Para ello, utilizaremos la función `G['nodes']`. Esta función nos regresa un diccionario entre el id y un array de indices de nuestros datos originales.

In [44]:
list(G['nodes'].keys())

['cube0_cluster0',
 'cube1_cluster0',
 'cube3_cluster0',
 'cube4_cluster0',
 'cube6_cluster0',
 'cube7_cluster0',
 'cube8_cluster0',
 'cube9_cluster0',
 'cube10_cluster0',
 'cube11_cluster0',
 'cube12_cluster0',
 'cube13_cluster0',
 'cube14_cluster0',
 'cube15_cluster0',
 'cube16_cluster0',
 'cube17_cluster0',
 'cube18_cluster0',
 'cube20_cluster0',
 'cube21_cluster0',
 'cube22_cluster0',
 'cube23_cluster0',
 'cube24_cluster0',
 'cube25_cluster0',
 'cube26_cluster0',
 'cube27_cluster0',
 'cube28_cluster0',
 0,
 'keys']

Por ejemplo las filas del cluster `cube1_cluster0`, son:

In [45]:
G['nodes']['cube1_cluster0']

[1,
 16,
 38,
 62,
 78,
 99,
 193,
 197,
 234,
 243,
 246,
 311,
 322,
 323,
 324,
 355,
 370,
 430,
 431,
 461]

In [54]:
df_close.iloc[:,G['nodes']['cube1_cluster0']].describe()

,AAL,AES,APA,BKR,CCL,CNP,GPS,HAL,IVZ,KEY,KIM,NI,NWL,NWS,NWSA,PPL,RF,UA,UAA,WMB
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000
mean,17.715262,20.165401,19.179096,19.897168,22.172234,21.990707,18.482150,19.270177,17.726821,16.845014,16.521611,23.160274,19.192910,18.262964,18.537946,25.933531,16.351518,15.135352,17.465493,20.742720
std,4.778093,4.588578,8.057639,5.569645,7.901045,3.884766,6.872308,6.663664,6.338960,4.780131,4.840809,2.384407,4.537905,5.139135,5.528692,2.351495,4.929670,4.294333,5.233328,4.654220
min,9.040000,8.718527,3.853528,8.466909,7.970000,11.186029,4.989502,4.449563,6.064301,7.189127,7.015776,18.637497,9.130189,7.884040,7.923611,16.147123,6.599072,6.890000,7.710000,7.682011
25%,13.275000,16.734096,13.282353,14.790091,16.645000,18.961913,13.565332,14.207729,10.483659,11.706394,11.435716,21.136932,15.300406,13.627635,13.628412,24.889928,11.389077,10.322500,11.742500,17.198881
50%,17.700000,21.708955,18.032022,20.437852,21.130000,22.714036,17.381608,19.901047,19.983146,17.802240,17.422751,23.086945,20.551197,20.857476,20.732196,26.203215,18.001540,16.040000,18.625000,20.086809
75%,21.012500,23.745456,24.913166,23.315624,25.172501,24.885059,22.879752,22.616728,23.435038,21.138221,20.549481,24.179614,22.862206,22.912836,23.198165,27.194496,20.651685,18.580000,21.690001,24.238838
max,30.469999,27.258400,41.202232,37.757092,51.301250,30.422235,33.224728,38.338284,27.550409,25.685110,24.337835,30.721687,26.712563,25.403221,27.053305,31.491776,24.333046,22.629999,26.959999,32.162971


In [56]:
df_close.iloc[:,G['nodes']['cube3_cluster0']].describe()

,BAC,BEN,BSX,CAG,CFG,CPB,CSX,DISH,DXC,EXC,...,NRG,OXY,PEAK,PFE,ROL,SLB,SYF,TPR,VZ,WY
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,...,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000
mean,33.093486,24.277131,39.921989,31.755930,35.448450,43.533609,28.729503,34.554771,27.564595,30.200290,...,33.725040,25.191930,28.790045,37.358080,32.579338,25.772460,34.554287,29.801230,49.808702,28.660174
std,8.611645,5.553532,3.832190,2.459573,10.540144,2.415585,5.110420,6.258551,8.239591,5.489404,...,4.382900,11.054212,4.032601,7.560578,4.743137,7.745893,10.720048,11.245193,1.920501,6.213552
min,16.880363,13.215375,25.830000,21.670155,13.422795,37.494255,15.310622,18.150000,9.129641,19.281139,...,19.544395,8.763958,17.507463,24.230362,20.345987,11.390889,11.814606,10.064469,42.467228,11.866326
25%,24.099223,19.117672,37.177500,30.604561,24.554371,41.883156,24.387126,31.190001,19.575879,25.611596,...,30.345979,15.554051,25.496007,32.067390,30.322031,18.327643,24.707271,16.701980,48.605627,25.040858
50%,32.841206,24.685643,40.700001,31.935498,38.698893,43.661427,29.807918,33.869999,29.590000,29.467738,...,33.963770,25.258230,29.989730,34.025301,33.839851,26.591846,35.510395,35.104614,49.914467,30.110250
75%,40.337502,29.232038,43.122499,33.450586,44.835821,45.334124,32.790208,39.925000,34.293086,33.296940,...,37.517746,31.020185,32.182925,41.789513,36.065827,31.564992,44.986548,39.432644,51.096801,33.544314
max,47.945427,35.271484,45.880001,36.176418,53.671524,49.156990,37.312683,46.529999,43.419998,46.165115,...,42.753494,60.423355,34.822605,58.783733,40.928082,44.347797,50.555378,46.726536,54.233448,38.985191


In [59]:
nodeid  = 'cube16_cluster0'
node = G['nodes'][nodeid]


plt.figure(figsize=(18, 8), dpi=80)
plt.rcParams.update({'font.size': 22})

for i in node:
    plt.plot(df_close.iloc[:,i], linewidth=2)
    
plt.legend(list(df_close.columns[node]), fontsize=18)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title(nodeid)

plt.savefig(nodeid+ ".png", dpi='figure', format=None, metadata=None,
        bbox_inches=None, pad_inches=0.1,
        facecolor='white', edgecolor='auto')